In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('fede'))
if module_path not in sys.path:
    sys.path.append(module_path)
import pandas as pd
from supported_modles import Supported_modles
import utils
from fedavg import Fedavg
from sklearn.utils.class_weight import compute_sample_weight
import numpy as np
from sklearn.metrics import f1_score
import random
from copy import deepcopy

In [2]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [3]:
clients, test_x, test_y = utils.set_data(True, False)

Creating node1.
Creating node2.
Creating node3.
Creating node4.
Creating node5.


In [4]:
selected_model = Supported_modles.MLP_classifier

In [5]:
f1_before = []
for client in clients:
    client.init_empty_model(selected_model,0.01)
    client.train_model()
    y_hat = client.model.predict(test_x)
    score = f1_score(test_y,y_hat, average="binary")
    f1_before.append(score)
    print(score)

0.9997627380034402
0.9995256307628451
0.999495952797462
0.9987998399786637
0.9993771872590308


In [6]:
fedavg = Fedavg("global", 0.1)

Waitiing for a Connection..


In [7]:
###First learn model on clients:
round_weights = []
dataset_size = 0
applicable_models = []
first = True

for client in clients:
    client.init_empty_model(selected_model,0.01)
    X_train, X_test, y_train, y_test = client.split_data()

    client.train_model(X_train,y_train)
    dataset_size += client.x.shape[0]
    round_weights.append(dataset_size)
    applicable_models.append(client.model)
    if first:
        fedavg.model = client.model
        first = False
    print('.')

round_weights = np.array(round_weights) / dataset_size # calculate weight based on actual dataset size
fedavg.update_global_model(applicable_models, round_weights, selected_model)

.
.
.
.
.


/home/tk/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:377: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a = np.asanyarray(a)


In [8]:
number_of_rounds = 21
batch_size = 0.2
epochs = 30
max_score = 0
optimal_model = None

for _ in range(number_of_rounds):
    print(_, end=' ')

    applicable_clients = random.sample((clients), random.randint(1, 4))
    applicable_models = []
    applicable_name = []
    round_weights = []
    dataset_size = 0
    

    for client in applicable_clients:
        # print(f'Client name: {client.name}')

        X_train, X_test, y_train, y_test = client.split_data()
       
        dataset_size += X_train.shape[0]
        sample_weights = compute_sample_weight('balanced', y=y_train)

        fedavg.load_global_model(client.model, selected_model)
        fedavg.train_local_agent(X_train, y_train, client.model, epochs, sample_weights, selected_model)
        round_weights.append(X_train.shape[0])
        applicable_models.append(client.model)


    round_weights = np.array(round_weights) / dataset_size # calculate weight based on actual dataset size
    fedavg.update_global_model(applicable_models, round_weights, selected_model)

    
    y_hat = fedavg.model.predict(test_x)
    score = f1_score(test_y,y_hat)
    if score > max_score:
        print(score)
        max_score = score
        optimal_model = deepcopy(fedavg.model)

0 

/home/tk/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:377: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a = np.asanyarray(a)


0.8961083574618443
1 

/home/tk/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:377: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a = np.asanyarray(a)


0.9983831971164543
2 

...

/home/tk/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:377: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a = np.asanyarray(a)


19 

/home/tk/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:377: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a = np.asanyarray(a)


20 

/home/tk/anaconda3/lib/python3.9/site-packages/numpy/lib/function_base.py:377: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a = np.asanyarray(a)


In [9]:
x = test_x
y_hat = optimal_model.predict(x)
score = f1_score(test_y,y_hat)
print(score)

0.9994513072234662


In [10]:
i = 0
for client in clients:
    print(f'Difference {score-f1_before[i]}')
    i += 1

Difference -0.0003114307799739757
Difference -7.432353937886216e-05
Difference -4.464557399574964e-05
Difference 0.0006514672448024728
Difference 7.411996443540403e-05


In [26]:
score

0.9578576199574382